In [ ]:
from scipy.stats import norm
import numpy as np
import pandas as pd


def bsCall(S, K, r, s, T):
    d1 = (np.log(S/K) + (r + 0.5*s**2)*T)/(s*np.sqrt(T))
    d2 = d1 - s*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    
def bsPut(S, K, r, s, T):
    d1 = (np.log(S/K) + (r + 0.5*s**2)*T)/(s*np.sqrt(T))
    d2 = d1 - s*np.sqrt(T)
    return -S*norm.cdf(-d1) + K*np.exp(-r*T)*norm.cdf(-d2)

vol = pd.read_csv('IV.csv', parse_dates=['Date']).iloc[1:, :].reset_index(drop=True)

In [ ]:
rates = pd.read_csv('LIBOR.csv', header=5, parse_dates=[0]).iloc[:, :2].reset_index(drop=True)
rates.columns = ['Date', 'LIBOR1M']

In [82]:
df = pd.merge(rates, vol, on='Date', how='inner').set_index('Date')
df = df.assign(SPX=pd.read_csv('data.csv', parse_dates=['Date'])[['Date', 'SPX']].set_index('Date').loc[df.index])
df.head()



,LIBOR1M,PUT_90,PUT_75,PUT_60,PUT_50,PUT_40,PUT_25,PUT_10,ATM,CALL_10,CALL_25,CALL_40,CALL_50,CALL_60,CALL_75,CALL_90,SPX
Date,,,,,,,,,,,,,,,,,
2005-01-04,2.40,13.4527,11.73719,11.80619,11.85213,12.50881,13.47850,14.38300,11.9063,11.66763,11.73713,11.80613,11.85188,12.50688,13.47681,14.38138,1188.05
2005-01-05,2.40,13.4918,11.79988,11.94081,12.03381,12.68488,13.65469,14.56019,12.0342,11.65950,11.80213,11.94288,12.03600,12.70050,13.66919,14.57369,1183.74
2005-01-06,2.42,12.9494,11.64731,11.57200,11.52163,12.08738,12.93950,13.73881,11.5200,11.72131,11.64650,11.57131,11.52081,12.09638,12.94788,13.74669,1187.89
2005-01-07,2.43,12.8366,11.32900,11.22613,11.15688,11.69213,12.49850,13.25600,11.1555,11.42988,11.32788,11.22488,11.15581,11.70131,12.50713,13.26413,1186.19
2005-01-10,2.44,12.6316,11.27913,11.14300,11.05131,11.77613,12.83050,13.80463,11.0877,11.41363,11.27919,11.14313,11.05150,11.77469,12.82950,13.80400,1190.24
